In [1]:
#!/usr/bin/env python

import random
import collections
import numpy as np
import pandas as pd
import csv

df = pd.read_csv('mbti_1.csv')

types = ('ISTJ', 'ISFJ', 'INFJ', 'INTJ', \
		 'ISTP', 'ISFP', 'INFP', 'INTP', \
		 'ESTP', 'ESFP', 'ENFP', 'ENTP', \
		 'ESTJ', 'ESFJ', 'ENFJ', 'ENTJ')

gen_pop = {'ISTJ': 0.11, 'ISFJ': 0.09, 'INFJ': 0.04, 'INTJ': 0.05, \
		   'ISTP': 0.05, 'ISFP': 0.05, 'INFP': 0.06, 'INTP': 0.06, \
		   'ESTP': 0.04, 'ESFP': 0.04, 'ENFP': 0.08, 'ENTP': 0.06, \
		   'ESTJ': 0.08, 'ESFJ': 0.09, 'ENFJ': 0.05, 'ENTJ': 0.05}

n, ___ = df.shape

counts = collections.defaultdict(int)
for mbti in df['type']:
	counts[mbti] += 1

limiting_type = None
min_size = float('infinity')
for mbti in counts.keys():
	size = counts[mbti]/gen_pop[mbti]
	if size < min_size:
		min_size = size
		limiting_type = mbti

dic = collections.defaultdict(list)
for row in df.iterrows():
	dic[row[1]['type']].append(row)

unclean_list = []


with open('mbti_clean.csv', 'w', encoding = 'UTF-8') as f:
	writer = csv.writer(f)
	writer.writerow(['type', 'posts'])
	
	for mbti in gen_pop.keys():
		list1 = dic[mbti]
		for x in range(0, int(round(min_size*gen_pop[mbti]))):
			writer.writerow(list1[x][1])	
		unclean_list.append(list1[int(round(min_size*gen_pop[mbti])) : len(list1)])
					
with open('mbti_unclean.csv', 'w', encoding = 'UTF-8') as f:
	writer = csv.writer(f)
	writer.writerow(['type', 'posts'])
	for mbti in unclean_list:
		for x in mbti:
			writer.writerow(x[1])


In [2]:

import random
import collections
import numpy as np
import pandas as pd
import csv
import re

df = pd.read_csv('mbti_unclean.csv')
DIMENSIONS = ('IE', 'NS', 'TF', 'PJ')
counts = collections.defaultdict(int)

for dimension in DIMENSIONS: 
	letter_1, letter_2 = dimension
	for mbti in df['type']:
		if letter_1 in mbti:
			counts[letter_1] += 1
		if letter_2 in mbti:
			counts[letter_2] += 1

for dimension in DIMENSIONS: 
	letter_1, letter_2 = dimension
	if counts[letter_1] < counts[letter_2]:
		limit = counts[letter_1]
	else: 
		limit = counts[letter_2]
	list1 = []
	list2 = []
	i = 0
	for row in df.iterrows():	
		if i == limit: break
		if letter_1 in row[1]['type']:
			list1.append(row[1]['posts'].split('|||'))
			i += 1
	i = 0
	for row in df.iterrows():	
		if i == limit: break
		if letter_2 in row[1]['type']:
			list2.append(row[1]['posts'].split('|||'))	
			i += 1
	with open('train_' + letter_1 + '.csv', 'w',encoding = 'UTF-8') as f:
		writer = csv.writer(f)
		for hundred_posts in list1:
			row = [post for post in hundred_posts if ('http' not in post) and (post != '') and (post != None) and (re.search("[a-zA-Z]", post))]
			writer.writerow(row)
	with open('train_' + letter_2 + '.csv', 'w',encoding = 'UTF-8') as f:
		writer = csv.writer(f)
		for hundred_posts in list2:
			row = [post for post in hundred_posts if ('http' not in post) and (post != '') and (post != None) and (re.search("[a-zA-Z]", post))]
			writer.writerow(row)


In [3]:
#!/usr/bin/env python

import random
import collections
import numpy as np
import pandas as pd
import csv
import re

letters = ('I', 'E', 'N', 'S', 'T', 'F', 'P', 'J')
df = pd.read_csv('mbti_clean.csv')
test = collections.defaultdict(list) 

for row in df.iterrows():
	posts = row[1]['posts'].split('|||')
	test[row[1]['type']].append(posts)

### write csv files for every every letter class and train vs. test class (16 total)
for letter in letters: 
	PATH = 'test_' + letter + '.csv'
	with open(PATH, 'w',encoding = 'UTF-8') as f:
		writer = csv.writer(f)
		for key in test.keys():
			if letter in key:
				for hundred_posts in test[key]:
					row = [post for post in hundred_posts if ('http' not in post) and (post != '') and (post != None) and (re.search("[a-zA-Z]", post))]
					if len(row) > 10: 
						writer.writerow(row)


In [4]:
#!/usr/bin/env python

import numpy as np
import pandas as pd
import csv
from sklearn.externals import joblib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.naive_bayes import MultinomialNB 
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier, Perceptron
from sklearn.neighbors import KNeighborsClassifier
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 

DIMENSIONS = ['IE', 'NS', 'FT', 'PJ']
CROSS_VALIDATION = False
SAVE_MODEL = False

for k in range(len(DIMENSIONS)):
	x_train = [] 
	y_train = [] 
	x_test = [] 
	y_test = []

	### Read in data
	with open('train_{}.csv'.format(DIMENSIONS[k][0]), 'r') as f: 
		reader = csv.reader(f)
		for row in reader:
			for post in row: 
				x_train.append(post)
				y_train.append(0)
	with open('train_{}.csv'.format(DIMENSIONS[k][1]), 'r') as f: 
		reader = csv.reader(f)
		for row in reader:
			for post in row: 
				x_train.append(post)
				y_train.append(1)
	with open('test_{}.csv'.format(DIMENSIONS[k][0]), 'r') as f: 
		reader = csv.reader(f)
		for row in reader:
			for post in row: 
				x_test.append(post)
				y_test.append(0)
	with open('test_{}.csv'.format(DIMENSIONS[k][1]), 'r') as f: 
		reader = csv.reader(f)
		for row in reader:
			for post in row: 
				x_test.append(post)
				y_test.append(1)

	types = ['INFJ', 'ENTP', 'INTP', 'INTJ', 'ENTJ', 'ENFJ', 'INFP', 'ENFP', 'ISFP', 'ISTP', 'ISFJ', 'ISTJ', 'ESTP', 'ESFP', 'ESTJ', 'ESFJ']
	types = [x.lower() for x in types]
	lemmatizer = WordNetLemmatizer()
	stop_words = stopwords.words("english")

	def lemmatize(x):
		lemmatized = []
		for post in x: 
			temp = post.lower() 
			for type_ in types: 
				temp = temp.replace(' ' + type_, '')
			temp = ' '.join([lemmatizer.lemmatize(word) for word in temp.split(' ') if (word not in stop_words)])
			lemmatized.append(temp)
		return np.array(lemmatized)
	def preprocess(x):
		lemmatized = lemmatize(x)
		tokenized = tokenizer.texts_to_sequences(lemmatized)
		return sequence.pad_sequences(tokenized, maxlen=MAX_POST_LENGTH)

	x_train = lemmatize(x_train)
	x_test = lemmatize(x_test)

	### Assign to dataframe
	df = pd.DataFrame(data={'text': x_train, 'type': y_train})
	df = df.sample(frac=1).reset_index(drop=True) ### Shuffle rows


	### Make pipeline
	pipeline = Pipeline([
		('vectorizer',  CountVectorizer(stop_words='english')), ### Bag-of-words
		('transformer', TfidfTransformer()), 
		('classifier',  MultinomialNB()) ]) ### Performs best
		#('classifier',  SVC()) ])
		#('classifier',  DecisionTreeClassifier(max_depth=50)) ])
		#('classifier', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=5, tol=None)) ])
		#('classifier',  Perceptron()) ])
		#('classifier',  KNeighborsClassifier(n_neighbors=2)) ])

	### Cross-validation classification (individual posts)
	if CROSS_VALIDATION: 
		k_fold = KFold(n=len(df), n_folds=6)
		scores_k = []
		confusion_k = np.array([[0, 0], [0, 0]])
		for train_indices, test_indices in k_fold:
			x_train_k = df.iloc[train_indices]['text'].values
			y_train_k = df.iloc[train_indices]['type'].values
			x_test_k = df.iloc[test_indices]['text'].values
			y_test_k = df.iloc[test_indices]['type'].values
			pipeline.fit(x_train_k, y_train_k)
			predictions_k = pipeline.predict(x_test_k)
			confusion_k += confusion_matrix(y_test_k, predictions_k)
			score_k = accuracy_score(y_test_k, predictions_k)
			scores_k.append(score_k)
		with open('cross_validation_{}.txt'.format(DIMENSIONS[k]), 'w') as f: 
			f.write('*** {}/{} TRAINING SET CROSS VALIDATION (POSTS) ***\n'.format(DIMENSIONS[k][0], DIMENSIONS[k][1]))
			f.write('Total posts classified: {}\n'.format(len(df)))
			f.write('Accuracy: {}\n'.format(sum(scores_k)/len(scores_k)))
			f.write('Confusion matrix: \n')
			f.write(np.array2string(confusion_k, separator=', '))

	### Test set classification (individual posts)
	pipeline.fit(df['text'].values, df['type'].values)
	predictions = pipeline.predict(x_test) 
	confusion = confusion_matrix(y_test, predictions)
	score = accuracy_score(y_test, predictions)
	with open('test_set_post_classification_{}.txt'.format(DIMENSIONS[k]), 'w') as f: 
		f.write('*** {}/{} TEST SET CLASSIFICATION (POSTS) ***\n'.format(DIMENSIONS[k][0], DIMENSIONS[k][1]))
		f.write('Total posts classified: {}\n'.format(len(x_test)))
		f.write('Accuracy: {}\n'.format(score))
		f.write('Confusion matrix: \n')
		f.write(np.array2string(confusion, separator=', '))

	###########################
	### USER CLASSIFICATION ###
	###########################
	
	### Read in user test set data
	x_test_a = []
	x_test_b = []
	with open('test_{}.csv'.format(DIMENSIONS[k][0]), 'r') as f: 
		reader = csv.reader(f)
		for row in reader:
			x_test_a.append(row)
	with open('test_{}.csv'.format(DIMENSIONS[k][1]), 'r') as f: 
		reader = csv.reader(f)
		for row in reader: 
			x_test_b.append(row)
	x_test = x_test_a + x_test_b

	### Make predictions for users
	'''
	predictions_a = []
	for user_batch in x_test_a:
		predictions_for_batch = pipeline.predict_classes(preprocess(user_batch))
		predictions_for_batch = [item for sublist in predictions_for_batch for item in sublist] ### Make flat list
		prediction = collections.Counter(predictions_for_batch).most_common(1)[0][0]
		predictions_a.append(prediction)
	predictions_b = []
	for user_batch in x_test_b:
		predictions_for_batch = pipeline.predict_classes(preprocess(user_batch))
		predictions_for_batch = [item for sublist in predictions_for_batch for item in sublist] ### Make flat list
		prediction = collections.Counter(predictions_for_batch).most_common(1)[0][0]
		predictions_b.append(prediction)
	predictions = predictions_a + predictions_b
	'''

	predictions = []
	for user in x_test: 
		prob = float(sum(pipeline.predict_proba(preprocess(user)))/len(user))
		prediction = 1
		if prob < 0.5: 
			prediction = 0
		predictions.append(prediction)

	### Analyze user classification results
	y_test_a = [0 for ___ in predictions_a]
	y_test_b = [1 for ___ in predictions_b]
	y_test = y_test_a + y_test_b
	confusion = confusion_matrix(y_test, predictions)
	score = accuracy_score(y_test, predictions)
	with open('test_set_user_classification_{}.txt'.format(DIMENSIONS[k]), 'w') as f: 
		f.write('*** {}/{} TEST SET CLASSIFICATION (USERS) ***\n'.format(DIMENSIONS[k][0], DIMENSIONS[k][1]))
		f.write('Total posts classified: {}\n'.format(len(x_test)))
		f.write('Accuracy: {}\n'.format(score))
		f.write('Confusion matrix: \n')
		f.write(np.array2string(confusion, separator=', '))

	# Save model
	if SAVE_MODEL:
		pipeline.named_steps['classifier'].model.save('NB_classifier_{}.h5'.format(DIMENSIONS[k]))
		pipeline.named_steps['classifier'].model = None
		joblib.dump(pipeline, 'NB_pipeline_{}.pkl'.format(DIMENSIONS[k]))
	del pipeline


/home/ajudi46/anaconda3/envs/pronew/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


NameError: name 'tokenizer' is not defined